### Практическое задание к уроку № 8 по теме "Дисперсионный анализ. Метод главных компонент. Логистическая регрессия".

#### Задание 1.  
*Используя данные о хоккеистах (скачать можно здесь), проверьте,  
является ли среди хоккеистов из Финляндии, Норвегии и Дании значимым отличие:  
а) роста, б) BMI.*

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

Загрузим датасет:

In [2]:
df = pd.read_csv('./hockey_players.csv', encoding='cp1252')
df.head()

,year,country,no,name,position,side,height,weight,birth,club,age,cohort,bmi
0,2001,RUS,10,tverdovsky oleg,D,L,185,84.0,1976-05-18,anaheim mighty ducks,24.952772,1976,24.543462
1,2001,RUS,2,vichnevsky vitali,D,L,188,86.0,1980-03-18,anaheim mighty ducks,21.119781,1980,24.332277
2,2001,RUS,26,petrochinin evgeni,D,L,182,95.0,1976-02-07,severstal cherepovetal,25.229295,1976,28.680111
3,2001,RUS,28,zhdan alexander,D,R,178,85.0,1971-08-28,ak bars kazan,29.675565,1971,26.827421
4,2001,RUS,32,orekhovsky oleg,D,R,175,88.0,1977-11-03,dynamo moscow,23.490760,1977,28.734694


Посмотрим уникальные значения стран:

In [3]:
df['country'].unique()

array(['RUS', 'AUT', 'BLR', 'CAN', 'CZE', 'FIN', 'GER', 'ITA', 'JPN',
       'LAT', 'NOR', 'SUI', 'SVK', 'SWE', 'UKR', 'USA', 'POL', 'SLO',
       'DEN', 'FRA', 'KAZ', 'HUN'], dtype=object)

Посмотрим на наличие дублирующих записей в датасете:

In [4]:
df[df.duplicated(subset=['birth', 'name', 'country'])]

,year,country,no,name,position,side,height,weight,birth,club,age,cohort,bmi
379,2002,RUS,25,zhukov sergei,D,L,193,92.0,1975-11-23,lokomotiv yaroslavl,26.436687,1975,24.698650
387,2002,RUS,13,razin andrei,F,L,181,83.0,1973-10-23,dynamo moskva,28.520192,1973,25.335002
390,2002,RUS,23,prokopiev alexander,F,R,181,85.0,1971-06-10,avangard omsk,30.891170,1971,25.945484
393,2002,RUS,27,gusmanov ravil,F,L,186,87.0,1972-07-25,metallurg magnitogorsk,29.765914,1972,25.147416
396,2002,RUS,44,karpov valeri,F,L,178,92.0,1971-08-05,metallurg magnitogorsk,30.737851,1971,29.036738
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6271,2016,USA,5,murphy connor,D,R,193,96.0,1993-03-26,arizona coyotes,23.099247,1993,25.772504
6276,2016,USA,11,nelson brock,F,L,191,93.0,1991-10-15,new york islanders,24.544832,1991,25.492722
6280,2016,USA,21,larkin dylan,F,L,185,86.0,1996-07-30,detroit red wings,19.753593,1996,25.127831
6281,2016,USA,23,hendricks matt,F,L,183,94.0,1981-06-17,edmonton oilers,34.872005,1981,28.068918


Удаляем дубликаты:

In [5]:
df = df.drop_duplicates(subset=['name', 'birth', 'country'])

Обозначим за переменные интересующие нас факторы по странам:

In [6]:
fin_height = np.array(df.loc[df['country'] == 'FIN', 'height'])
nor_height = np.array(df.loc[df['country'] == 'NOR', 'height'])
den_height = np.array(df.loc[df['country'] == 'DEN', 'height'])

fin_bmi = np.array(df.loc[df['country'] == 'FIN', 'bmi'])
nor_bmi = np.array(df.loc[df['country'] == 'NOR', 'bmi'])
den_bmi = np.array(df.loc[df['country'] == 'DEN', 'bmi'])

Будем использовать дисперсионный анализ. За нулевую гипотезу примем то,  
что страна происхождения хоккеиста не влияет на значение целевого признака  
(рост, BMI).  
Зададим уровень значимости для проверки гипотезы:

In [7]:
alpha = 0.05

Весь расчёт обозначим внутри функции, чтобы не дублировать код при  
выборе другого целевого признака:

In [8]:
def analyze(y1, y2, y3):
    
    # Подготавливаем данные
    samples = (y1, y2, y3)
    Y = np.concatenate([y1, y2, y3])
    
    # Считаем суммы квадратов отклонений между группами и 
    # внутри групп для дальнейшего расчёта
    SSb = sum((yi.mean() - Y.mean())**2 * yi.shape[0] for yi in samples)
    SSw = sum(sum((yi - yi.mean())**2) for yi in samples)

    # Расчёт соответствующих дисперсий для расчёта F-статистики
    k = len(samples)
    n = Y.shape[0]
    sigma2_b = SSb / (k - 1)
    sigma2_w = SSw / (n - k)

    # Расчёт F-статистики
    F = sigma2_b / sigma2_w
    print(f'Статистика F: {F}')

    # Нахождение критической области при распределении Фишера
    k1 = k - 1
    k2 = n - k
    t = stats.f.ppf(1-alpha, k1, k2)
    print(f'k1: {k1}, k2: {k2}')
    print(f'Критическая область: ({t}, +inf)')

    # Вывод о принятии / непринятии нулевой гипотезы
    my_str = ''
    if F < t:
        my_str = 'НЕ '
    
    print(f'\nЗначение статистики {my_str}попало в критическую область, ' \
          f'значит для хоккеистов из этих стран \nотличие по этому фактору ' \
          f'{my_str}является значимым.')


Проверим значимость по фактору роста:

In [9]:
analyze(fin_height, nor_height, den_height)

Статистика F: 3.5934575072406427
k1: 2, k2: 430
Критическая область: (3.016700272452327, +inf)

Значение статистики попало в критическую область, значит для хоккеистов из этих стран 
отличие по этому фактору является значимым.


Значимость по фактору BMI:

In [10]:
analyze(fin_bmi, nor_bmi, den_bmi)

Статистика F: 8.68251342053196
k1: 2, k2: 430
Критическая область: (3.016700272452327, +inf)

Значение статистики попало в критическую область, значит для хоккеистов из этих стран 
отличие по этому фактору является значимым.


<u>Вывод:</u> по обоим факторам отличия являются значимыми.